# Model 1: Klasifikasi Aktivitas

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

In [14]:
paths = [
    "guts_dataset.csv", "health_dataset.csv", "kindness_dataset.csv",
    "knowledge_dataset.csv", "proficiency_dataset.csv", "charm_dataset.csv"
]
paths = [f"E:/Coding/persona-ai/datasets/{p}" for p in paths]

df = pd.concat([pd.read_csv(p) for p in paths], ignore_index=True)
df.dropna(inplace=True)

df.tail()

,text_aktivitas,kategori,tingkat_aktivitas
1877,Saya berusaha untuk memberi perhatian lebih ke...,Charm,Berat
1878,Saya berbicara dengan nada suara yang menyenan...,Charm,Ringan
1879,Saya berusaha untuk mengerti perasaan orang la...,Charm,Berat
1880,Saya berusaha untuk mengerti perasaan orang la...,Charm,Ringan
1881,Saya mencoba untuk menjadi orang yang menyenan...,Charm,Ringan


In [15]:
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["kategori"])
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

texts = df["text_aktivitas"].tolist()
labels = df["label"].tolist()

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=42
)

In [17]:
vectorizer = TfidfVectorizer(max_features=1000)
X_vec = vectorizer.fit_transform(X_train)

smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_vec, y_train)

texts_resampled = [" ".join(words) for words in vectorizer.inverse_transform(X_res)]
labels_resampled = y_res

In [18]:
# Tokenize (train) with IndoBERT tokenizer

tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')

train_encodings = tokenizer(
    texts_resampled, truncation=True, padding=True, max_length=128, return_tensors="tf"
)
train_labels_tensor = tf.convert_to_tensor(labels_resampled)

In [19]:
# Tokenize (test)

test_encodings = tokenizer(
    X_test, truncation=True, padding=True, max_length=128, return_tensors="tf"
)
test_labels_tensor = tf.convert_to_tensor(y_test)

In [ ]:
# Train with Stratified K-Fold (on train only)

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(train_labels_tensor)), train_labels_tensor)):
    print(f"\n🚀 Fold {fold + 1}")

    train_inputs = {key: tf.gather(val, train_idx) for key, val in train_encodings.items()}
    val_inputs = {key: tf.gather(val, val_idx) for key, val in train_encodings.items()}
    train_labels = tf.gather(train_labels_tensor, train_idx)
    val_labels = tf.gather(train_labels_tensor, val_idx)

    model = TFBertForSequenceClassification.from_pretrained(
        'indobenchmark/indobert-base-p1', num_labels=len(label_mapping)
    )

    optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy()

    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

    model.fit(
        train_inputs,
        train_labels,
        validation_data=(val_inputs, val_labels),
        epochs=3,
        batch_size=16
    )

    model.save_pretrained(f"personaai_bert_fold{fold+1}")
    tokenizer.save_pretrained(f"personaai_bert_fold{fold+1}")
    print(f"✅ Model untuk fold {fold + 1} disimpan.")

## Evaluasi Klasifikasi

In [21]:
print("\n📊 Evaluasi model terakhir pada test set")

final_model = TFBertForSequenceClassification.from_pretrained("personaai_bert_fold3")
final_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

test_results = final_model.evaluate(test_encodings, test_labels_tensor)
print(f"\n🎯 Test Accuracy: {test_results[1]*100:.2f}%")


📊 Evaluasi model terakhir pada test set


Some layers from the model checkpoint at personaai_bert_fold3 were not used when initializing TFBertForSequenceClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at personaai_bert_fold3.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


12/12 [==============================] - 6s 392ms/step - loss: 0.0525 - sparse_categorical_accuracy: 0.9894

🎯 Test Accuracy: 98.94%


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Ambil prediksi logit dan ubah ke label prediksi
logits = final_model.predict(test_encodings).logits
y_pred = np.argmax(logits, axis=1)
y_true = y_test  # Sudah dalam format list

# Classification Report
print("\n📄 Classification Report")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))

# Confusion Matrix
print("📉 Confusion Matrix")
print(confusion_matrix(y_true, y_pred))

# Jika ingin metrik satuan:
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

print(f"\n📌 Detail Metrics:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-Score : {f1:.4f}")


12/12 [==============================] - 7s 404ms/step

📄 Classification Report
              precision    recall  f1-score   support

       Charm       1.00      0.98      0.99        50
        Guts       1.00      1.00      1.00        44
      Health       0.98      1.00      0.99        60
    Kindness       1.00      1.00      1.00        45
   Knowledge       0.97      1.00      0.98        98
 Proficiency       1.00      0.96      0.98        80

    accuracy                           0.99       377
   macro avg       0.99      0.99      0.99       377
weighted avg       0.99      0.99      0.99       377

📉 Confusion Matrix
[[49  0  1  0  0  0]
 [ 0 44  0  0  0  0]
 [ 0  0 60  0  0  0]
 [ 0  0  0 45  0  0]
 [ 0  0  0  0 98  0]
 [ 0  0  0  0  3 77]]

📌 Detail Metrics:
Accuracy : 0.9894
Precision: 0.9897
Recall   : 0.9894
F1-Score : 0.9894


In [25]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

# =======================
# Load Model dan Tokenizer
# =======================
model_path = "personaai_bert_fold1"  # pastikan folder ini sudah ada dan berisi model/tokenizer hasil training

model = TFBertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

# Mapping label sesuai training-mu
label_mapping = {
    0: "charm",
    1: "guts",
    2: "health",
    3: "kindness",
    4: "knowledge",
    5: "proficiency"
}
inv_label_mapping = {v: k for k, v in label_mapping.items()}

# =======================
# Fungsi Prediksi
# =======================
def predict_category(text):
    encoding = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="tf"
    )
    logits = model(encoding).logits
    pred = tf.argmax(logits, axis=1).numpy()[0]
    return label_mapping[pred]

# =======================
# Interaktif di Notebook
# =======================
from IPython.display import display, Markdown

def run_manual_prediction():
    while True:
        text = input("📥 Masukkan teks aktivitas (atau ketik 'exit' untuk keluar): ")
        if text.lower() == "exit":
            print("👋 Terima kasih, selesai.")
            break
        pred = predict_category(text)
        display(Markdown(f"**🎯 Prediksi Kategori: `{pred.upper()}`**\n"))

# Jalankan fungsi interaktif:
run_manual_prediction()

Some layers from the model checkpoint at personaai_bert_fold1 were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at personaai_bert_fold1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


**🎯 Prediksi Kategori: `HEALTH`**


**🎯 Prediksi Kategori: `PROFICIENCY`**


**🎯 Prediksi Kategori: `KINDNESS`**


**🎯 Prediksi Kategori: `CHARM`**


**🎯 Prediksi Kategori: `KINDNESS`**


👋 Terima kasih, selesai.


# Model 2: Klasifikasi Tingkat Aktivitas

In [64]:
paths = [
    "guts_dataset.csv", "health_dataset.csv", "kindness_dataset.csv",
    "knowledge_dataset.csv", "proficiency_dataset.csv", "charm_dataset.csv"
]
paths = [f"E:/Coding/persona-ai/datasets/{p}" for p in paths]
df = pd.concat([pd.read_csv(p) for p in paths], ignore_index=True)
df.dropna(inplace=True)

df.head()

,text_aktivitas,kategori,tingkat_aktivitas
0,Saya keluar dari zona nyaman saya dan meningga...,Guts,Menengah
1,Saya berani mengubah hidup saya dengan berpind...,Guts,Berat
2,Saya mengambil risiko besar dalam hidup saya k...,Guts,Ringan
3,Saya percaya bahwa langkah besar saya akan mem...,Guts,Ringan
4,Saya melangkah maju meskipun saya merasa takut...,Guts,Menengah


In [65]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df["label_xp"] = label_encoder.fit_transform(df["tingkat_aktivitas"])
label_mapping_xp = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

texts = df["text_aktivitas"].tolist()
labels = df["label_xp"].tolist()

In [66]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

ros = RandomOverSampler(random_state=42)
df_text_label = pd.DataFrame({"text": texts, "label": labels})
texts_resampled, labels_resampled = ros.fit_resample(df_text_label[["text"]], df_text_label["label"])
texts_resampled = texts_resampled["text"].tolist()

print("✅ Distribusi setelah resampling:", Counter(labels_resampled))

✅ Distribusi setelah resampling: Counter({1: 934, 0: 934, 2: 934})


In [67]:
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
train_encodings = tokenizer(
    texts_resampled, truncation=True, padding=True, max_length=128, return_tensors="tf"
)
train_labels_tensor = tf.convert_to_tensor(labels_resampled)

In [68]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(train_labels_tensor)), train_labels_tensor)):
    print(f"\n🚀 Fold {fold + 1}")

    train_inputs = {key: tf.gather(val, train_idx) for key, val in train_encodings.items()}
    val_inputs = {key: tf.gather(val, val_idx) for key, val in train_encodings.items()}
    train_labels = tf.gather(train_labels_tensor, train_idx)
    val_labels = tf.gather(train_labels_tensor, val_idx)


🚀 Fold 1

🚀 Fold 2

🚀 Fold 3


In [70]:
model2 = TFBertForSequenceClassification.from_pretrained(
    'indobenchmark/indobert-base-p1', num_labels=len(label_mapping_xp)
)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy()

model2.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model2.fit(
    train_inputs, train_labels,
    validation_data=(val_inputs, val_labels),
    epochs=15,
    batch_size=16,
)

save_path = f"personaai_xp_fold{fold+1}"
model2.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"✅ Model tingkat aktivitas untuk fold {fold + 1} disimpan di {save_path}")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/15
117/117 [==============================] - 159s 1s/step - loss: 1.1185 - sparse_categorical_accuracy: 0.3469 - val_loss: 1.0968 - val_sparse_categorical_accuracy: 0.3319
Epoch 2/15
117/117 [==============================] - 138s 1s/step - loss: 1.0827 - sparse_categorical_accuracy: 0.4074 - val_loss: 1.0587 - val_sparse_categorical_accuracy: 0.4486
Epoch 3/15
117/117 [==============================] - 138s 1s/step - loss: 0.9829 - sparse_categorical_accuracy: 0.5091 - val_loss: 1.0319 - val_sparse_categorical_accuracy: 0.4872
Epoch 4/15
117/117 [==============================] - 138s 1s/step - loss: 0.8516 - sparse_categorical_accuracy: 0.6146 - val_loss: 1.0003 - val_sparse_categorical_accuracy: 0.5343
Epoch 5/15
117/117 [==============================] - 137s 1s/step - loss: 0.6976 - sparse_categorical_accuracy: 0.7061 - val_loss: 1.0345 - val_sparse_categorical_accuracy: 0.5418
Epoch 6/15
117/117 [==============================] - 138s 1s/step - loss: 0.5750 - sparse_cate

In [71]:
print("\n📊 Evaluasi model terakhir pada test set")

final2 = TFBertForSequenceClassification.from_pretrained("personaai_xp_fold3")
final2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

test_results = final2.evaluate(test_encodings, test_labels_tensor)
print(f"\n🎯 Test Accuracy: {test_results[1]*100:.2f}%")


📊 Evaluasi model terakhir pada test set


Some layers from the model checkpoint at personaai_xp_fold3 were not used when initializing TFBertForSequenceClassification: ['dropout_1025']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at personaai_xp_fold3.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


12/12 [==============================] - 7s 401ms/step - loss: 0.6737 - sparse_categorical_accuracy: 0.8462

🎯 Test Accuracy: 84.62%


In [72]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Define a function for model evaluation
def evaluate_model(model, test_encodings, y_test, label_encoder, fold_name="Current"):
    """
    Comprehensive model evaluation function that handles warnings properly
    and provides detailed visualizations.
    """
    print(f"\n{'='*60}")
    print(f"Evaluating {fold_name} Model")
    print(f"{'='*60}")
    
    # Get prediction logits and convert to predicted labels
    logits = model.predict(test_encodings).logits
    y_pred = np.argmax(logits, axis=1)
    y_true = y_test  # Already in list format
    
    # Classification Report
    print("\n📄 Classification Report")
    class_report = classification_report(y_true, y_pred, 
                                        target_names=label_encoder.classes_,
                                        zero_division=0)
    print(class_report)
    
    # Confusion Matrix
    print("\n📉 Confusion Matrix")
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    
    # Calculate metrics with proper handling of zero divisions
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    print(f"\n📌 Detail Metrics:")
    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1-Score : {f1:.4f}")
    
    # Additional analysis: class distribution
    print("\n📊 Class Distribution Analysis")
    class_count_true = np.bincount(y_true)
    class_count_pred = np.bincount(y_pred)
    
    # Handle case where bincount dimensions don't match number of classes
    if len(class_count_true) < len(label_encoder.classes_):
        class_count_true = np.pad(class_count_true, 
                                 (0, len(label_encoder.classes_) - len(class_count_true)))
    if len(class_count_pred) < len(label_encoder.classes_):
        class_count_pred = np.pad(class_count_pred, 
                                 (0, len(label_encoder.classes_) - len(class_count_pred)))
    
    for i, class_name in enumerate(label_encoder.classes_):
        print(f"{class_name}: {class_count_true[i]} actual samples, {class_count_pred[i]} predicted samples")
    
    
    # Find problematic classes (where no samples were predicted)
    problematic_classes = [label_encoder.classes_[i] 
                          for i in range(len(label_encoder.classes_)) 
                          if i < len(class_count_pred) and class_count_pred[i] == 0]
    
    if problematic_classes:
        print("\n⚠️ Warning: The following classes have no predicted samples:")
        for cls in problematic_classes:
            print(f"- {cls}")
        print("This can lead to ill-defined precision for these classes.")
    
    # Calculate per-class metrics
    print("\n📍 Per-Class Metrics:")
    per_class_precision = precision_score(y_true, y_pred, average=None, zero_division=0)
    per_class_recall = recall_score(y_true, y_pred, average=None, zero_division=0)
    per_class_f1 = f1_score(y_true, y_pred, average=None, zero_division=0)
    
    # Make sure metrics arrays match the number of classes
    if len(per_class_precision) < len(label_encoder.classes_):
        per_class_precision = np.pad(per_class_precision, 
                                   (0, len(label_encoder.classes_) - len(per_class_precision)))
    if len(per_class_recall) < len(label_encoder.classes_):
        per_class_recall = np.pad(per_class_recall, 
                                (0, len(label_encoder.classes_) - len(per_class_recall)))
    if len(per_class_f1) < len(label_encoder.classes_):
        per_class_f1 = np.pad(per_class_f1, 
                             (0, len(label_encoder.classes_) - len(per_class_f1)))
    
    # Create a DataFrame for per-class metrics
    per_class_df = pd.DataFrame({
        'Class': label_encoder.classes_,
        'Precision': per_class_precision[:len(label_encoder.classes_)],
        'Recall': per_class_recall[:len(label_encoder.classes_)],
        'F1-Score': per_class_f1[:len(label_encoder.classes_)]
    })
    
    print(per_class_df)

# To evaluate a single model:
results = evaluate_model(model2, test_encodings, y_test, label_encoder, "Model 2")

# To evaluate all fold models together:
def evaluate_all_folds(fold_models, test_encodings, y_test, label_encoder):
    """
    Evaluate all fold models and compare their performance
    """
    all_results = []
    
    # Evaluate each fold model
    for i, model in enumerate(fold_models):
        print(f"\nEvaluating Fold {i+1} Model")
        result = evaluate_model(model, test_encodings, y_test, label_encoder, f"Fold {i+1}")
        result['fold'] = i+1
        all_results.append(result)
    
    # Create ensemble predictions (majority voting)
    print("\n\n" + "="*60)
    print("Ensemble Model Evaluation (Majority Voting)")
    print("="*60)
    
    ensemble_predictions = []
    
    # For each test sample
    for i in range(len(y_test)):
        # Get predictions from all folds
        fold_votes = [all_results[j]['predictions'][i] for j in range(len(fold_models))]
        # Take the most common prediction (majority vote)
        from collections import Counter
        most_common = Counter(fold_votes).most_common(1)[0][0]
        ensemble_predictions.append(most_common)
    
    # Evaluate ensemble predictions
    acc = accuracy_score(y_test, ensemble_predictions)
    prec = precision_score(y_test, ensemble_predictions, average='weighted', zero_division=0)
    rec = recall_score(y_test, ensemble_predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test, ensemble_predictions, average='weighted', zero_division=0)
    
    print(f"\n📌 Ensemble Model Metrics:")
    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1-Score : {f1:.4f}")
    
    # Compare fold models
    fold_comparison = pd.DataFrame([
        {
            'Fold': result['fold'],
            'Accuracy': result['accuracy'],
            'Precision': result['precision'],
            'Recall': result['recall'],
            'F1 Score': result['f1']
        }
        for result in all_results
    ])
    
    # Add ensemble row
    fold_comparison = pd.concat([
        fold_comparison,
        pd.DataFrame([{
            'Fold': 'Ensemble',
            'Accuracy': acc,
            'Precision': prec,
            'Recall': rec,
            'F1 Score': f1
        }])
    ])
    
    print("\n📊 Fold Performance Comparison:")
    print(fold_comparison)
    
    # Visualize fold comparison
    plt.figure(figsize=(12, 6))
    fold_comparison.set_index('Fold').plot(kind='bar')
    plt.title('Performance Comparison Across Folds')
    plt.ylabel('Score')
    plt.ylim(0, 1.1)
    plt.legend(loc='lower right')
    plt.tight_layout()
    plt.show()
    
    return {
        'fold_results': all_results,
        'ensemble_accuracy': acc,
        'ensemble_precision': prec,
        'ensemble_recall': rec,
        'ensemble_f1': f1,
        'ensemble_predictions': ensemble_predictions,
        'fold_comparison': fold_comparison
    }

# Example usage:
# Define fold_models first
# fold_models = [model1, model2, model3]  # Your three fold models
# ensemble_results = evaluate_all_folds(fold_models, test_encodings, y_test, label_encoder)


Evaluating Model 2 Model
12/12 [==============================] - 7s 381ms/step

📄 Classification Report
              precision    recall  f1-score   support

       Berat       0.80      0.96      0.88        77
    Menengah       0.78      0.87      0.82       113
      Ringan       0.92      0.79      0.85       187

    accuracy                           0.85       377
   macro avg       0.84      0.87      0.85       377
weighted avg       0.85      0.85      0.85       377


📉 Confusion Matrix
[[ 74   1   2]
 [  4  98  11]
 [ 14  26 147]]

📌 Detail Metrics:
Accuracy : 0.8462
Precision: 0.8550
Recall   : 0.8462
F1-Score : 0.8460

📊 Class Distribution Analysis
Berat: 77 actual samples, 92 predicted samples
Menengah: 113 actual samples, 125 predicted samples
Ringan: 187 actual samples, 160 predicted samples

📍 Per-Class Metrics:
      Class  Precision    Recall  F1-Score
0     Berat   0.804348  0.961039  0.875740
1  Menengah   0.784000  0.867257  0.823529
2    Ringan   0.918750  0

## Prediksi Ganda: Kategori + Tingkat Aktivitas

In [73]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from IPython.display import display, Markdown

# =======================
# Load Model dan Tokenizer
# =======================
# Model 1: Prediksi Kategori
kategori_model_path = "personaai_bert_fold1"
model_kategori = TFBertForSequenceClassification.from_pretrained(kategori_model_path)
tokenizer_kategori = BertTokenizer.from_pretrained(kategori_model_path)

# Model 2: Prediksi Tingkat Aktivitas
aktivitas_model_path = "personaai_xp_fold3"
model_aktivitas = TFBertForSequenceClassification.from_pretrained(aktivitas_model_path)
tokenizer_aktivitas = BertTokenizer.from_pretrained(aktivitas_model_path)

# =======================
# Mapping Label
# =======================
label_mapping_kategori = {
    0: "charm",
    1: "guts",
    2: "health",
    3: "kindness",
    4: "knowledge",
    5: "proficiency"
}

label_encoder_aktivitas = LabelEncoder()
label_encoder_aktivitas.fit(["Berat", "Menengah", "Ringan"])

# =======================
# Fungsi Prediksi
# =======================
def predict_all(text):
    # Prediksi kategori
    enc_kat = tokenizer_kategori(text, truncation=True, padding=True, max_length=128, return_tensors="tf")
    logits_kat = model_kategori(enc_kat).logits
    pred_kat_id = tf.argmax(logits_kat, axis=1).numpy()[0]
    pred_kat_label = label_mapping_kategori[pred_kat_id]

    # Prediksi tingkat aktivitas
    enc_akt = tokenizer_aktivitas(text, truncation=True, padding=True, max_length=128, return_tensors="tf")
    logits_akt = model_aktivitas(enc_akt).logits
    pred_akt_id = tf.argmax(logits_akt, axis=1).numpy()[0]
    pred_akt_label = label_encoder_aktivitas.inverse_transform([pred_akt_id])[0]

    return pred_kat_label, pred_akt_label

# =======================
# Fungsi Interaktif
# =======================
def run_double_prediction():
    while True:
        text = input("📥 Masukkan teks aktivitas (atau ketik 'exit' untuk keluar): ")
        if text.lower() == "exit":
            print("👋 Terima kasih, selesai.")
            break
        pred_kat, pred_akt = predict_all(text)
        display(Markdown(f"""
**🎯 Prediksi Kategori:** `{pred_kat.upper()}`
**📌 Prediksi Tingkat Aktivitas:** `{pred_akt.upper()}`
        """))

# Jalankan fungsi:
run_double_prediction()


Some layers from the model checkpoint at personaai_bert_fold1 were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at personaai_bert_fold1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
Some layers from the model checkpoint at personaai_xp_fold3 were not 


**🎯 Prediksi Kategori:** `HEALTH`
**📌 Prediksi Tingkat Aktivitas:** `MENENGAH`
        


**🎯 Prediksi Kategori:** `PROFICIENCY`
**📌 Prediksi Tingkat Aktivitas:** `MENENGAH`
        


**🎯 Prediksi Kategori:** `PROFICIENCY`
**📌 Prediksi Tingkat Aktivitas:** `MENENGAH`
        


**🎯 Prediksi Kategori:** `HEALTH`
**📌 Prediksi Tingkat Aktivitas:** `MENENGAH`
        

...


**🎯 Prediksi Kategori:** `KNOWLEDGE`
**📌 Prediksi Tingkat Aktivitas:** `MENENGAH`
        


**🎯 Prediksi Kategori:** `KNOWLEDGE`
**📌 Prediksi Tingkat Aktivitas:** `BERAT`
        


**🎯 Prediksi Kategori:** `PROFICIENCY`
**📌 Prediksi Tingkat Aktivitas:** `MENENGAH`
        


**🎯 Prediksi Kategori:** `KINDNESS`
**📌 Prediksi Tingkat Aktivitas:** `MENENGAH`
        

👋 Terima kasih, selesai.
